In [ ]:
# Loading data from csv file
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/files/lsh_assignment_data.csv")
#df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [ ]:
print(train_data.shape)
print(test_data.shape)

(2215, 2)
(10, 2)


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
from numpy import dot
from numpy.linalg import norm

#Train Data

In [ ]:
#stop= set(stopwords.words('english'))
vect=TfidfVectorizer(max_features=4000,ngram_range=(2,3),min_df=10)
vect.fit(np.array(train_data["text"]))
train_vec=vect.transform(np.array(train_data["text"]))

Generate 5 random hyperplanes. And generate the hyperplanes using a random normal distribution with mean zero and variance 1

In [ ]:
np.random.seed(0)
hyperplane=[]
for i in range(5):
  hyperplane.append(list(np.random.normal(loc=0,scale=1,size=train_vec.shape[1])))
hyperplane=np.array(hyperplane)
#print(hyperplane)

Hash Function, Hash Table, hash_key

In [ ]:
Hash_table=dict()
for idx,train_data_pnt in enumerate(train_vec.toarray()):
  Hash_key=tuple([1 if 0<=np.dot(W,train_data_pnt) else 0 for W in hyperplane])
  #-------------------------
  if Hash_key in Hash_table:
    #----this will be a list
    Hash_table[Hash_key].append(idx)
  else:
    Hash_table[Hash_key]=list([idx])

len(Hash_table)

32

#Test Data

In [ ]:
test_vec=vect.transform(np.array(test_data["text"]))

Getting similar data points using hash table for every test data point (0-9)

In [ ]:
similar_points_idx=[]
for idx, point in enumerate(test_vec.toarray()):
  test_hash_key=tuple([1 if 0.0<=np.dot(point,W) else 0 for W in hyperplane])
  if test_hash_key in Hash_table:
    similar_points_idx.append(Hash_table[test_hash_key])
  else:
    pass

Cosine similarity between test data point and all the train points in bucket

In [ ]:
consine_similarities=[]
for i,lst in enumerate(similar_points_idx):
  temp=[]
  for idx in lst:
    cos_sim=dot(test_vec.toarray()[i],train_vec.toarray()[idx])/(norm(test_vec.toarray()[i])*norm(train_vec.toarray()[idx]))
    temp.append(cos_sim)
  consine_similarities.append(temp)

here List1 will get sorted in decreasing order based on the list2, in output you will get List1 sorted based on list2


In [ ]:
similar_pts_sorted=[[element for _, element in sorted(zip(list2,list1),reverse=True)][0:11] for list1,list2 in zip(similar_points_idx ,consine_similarities)]
#print(similar_pts_sorted)

class labels for nearest 11 train dataset based on cosine similarity

In [ ]:
class_labels=[]
for lst in similar_pts_sorted:
  temp=[]
  for idx in lst:
    temp.append(train_data["category"].iloc[idx])
  class_labels.append(temp)
#print(class_labels)

 Majority vote and predicted class label


In [ ]:
Predicted_class_label=[max(sorted(lst),key=sorted(lst).count) for lst in class_labels]
print(Predicted_class_label)

['tech', 'entertainment', 'tech', 'tech', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport']


In [2]:
test_vec

NameError: ignored

#Grader
Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.



In [ ]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the computeTFIDF() is correct.
# Else, it will print "Failed"
###########################################
import numpy as np

# compute TF-IDF using the computeTFIDF() function
#Y_custom = np.array(predictLabels(test_data))
Y_custom=Predicted_class_label

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

#calculating accuracy by comparing Y_grader and Y_custom
accuracy=np.sum(Y_grader==Y_custom)*10
# compare Y_grader and Y_custom
print(accuracy)
if accuracy>=80:
  print("******** Success ********","Accuracy Achieved",accuracy,'%')
else:
  print("####### Failed #######")
  print("\Y_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\Y_custom = \n\n", Y_custom)


90
******** Success ******** Accuracy Achieved 90 %
